In [ ]:
import os
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List
import pandas as pd

# Function to compute angle (assuming your existing implementation)
def compute_angle(A, B, C):
    # Your existing compute_angle function
    # ...

# Function to calculate pointer angle from an image
def calculate_pointer_angle(img):
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

# Function to process an image and calculate angles
def process_image(img_path):
    image = cv2.imread(img_path)
    if image is None:
        print(f"Unable to read image: {img_path}")
        return None, None
    return calculate_pointer_angle(image)

# Function to save results to Excel
def save_to_excel(file_name, angle_ABC, angle_BDC, excel_output_path):
    data = {'File': [file_name], 'Angle_ABC': [angle_ABC], 'Angle_BDC': [angle_BDC]}
    df = pd.DataFrame(data)
    
    if not os.path.isfile(excel_output_path):
        df.to_excel(excel_output_path, index=False)
    else:  # else it exists so append without writing the header
        with pd.ExcelWriter(excel_output_path, engine='openpyxl', mode='a') as writer:
            df.to_excel(writer, index=False, header=False)

if __name__ == "__main__":
    model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'
    folder_path = "/path/to/your/image/folder"
    excel_output_path = 'angles_results.xlsx'

    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

    if not image_files:
        print("No image files found in the specified folder.")
        sys.exit()

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        image_output = process_image(img_path)

        if image_output is not None:
            angle_ABC, angle_BDC = image_output
            save_to_excel(img_file, angle_ABC, angle_BDC, excel_output_path)

    print(f"Results saved to {excel_output_path}")
